## Disecting Model 01

@ Anne
* use `random` instead of `numpy.random`
* `random.choices` instead of if-else
* for cell in grid_iter: _, (x,y) (directly)
* don't import single functions when they can be from any of the libraries
* adding leading zeros (see zfill)
* ifelse to delete existing files, or add datetime to subfolder name

In [1]:
# import packages
# import mesa.space
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random
import numpy as np
import statistics
import matplotlib.pyplot as plt
import math
import os
import cv2

**Define global level functions**

**Agent class**

In [2]:
# define Agent class
class SchellingAgent(Agent):
    """
    Schelling segregation agent
    """

    def __init__(self, unique_id, pos, model, agent_type, income):
        """
        Create a new Schelling agent.

        Args:
           unique_id: Unique identifier for the agent.
           x, y: Agent initial location.
           agent_type: Indicator for the agent's type (minority=1, majority=0)
           
        """
        super().__init__(pos, model)
        self.pos = pos
        self.type = agent_type
        self.income = income
        self.unique_id = unique_id

    ''' Below step is completely changed compared to the other/old models (NEW) '''
    def step(self):
        average_parcel_income = self.model.parcel_values[self.pos]

        #If own income is higher than average income of current parcel
        if self.income > average_parcel_income:
            #print(f'this is the income: {self.income}')
            old_location = self.pos

            parcel_list = []

            if self.type == 0:
                #Look for parcels where they have on average a higher income than own income
                for key, value in self.model.parcel_values.items():
                    #print(f'this is the value: {value}')
                    if value > self.income or value ==0:
                        parcel_list.append(key)

            else:
                # Look for parcels where they have on average a lower income than own income
                for key, value in self.model.parcel_values.items():
                    # print(f'this is the value: {value}')
                    if value < self.income:
                        parcel_list.append(key)

            #If there is a 'wealthier' parcel, move to a randomly chosen one
            if len(parcel_list) != 0:
                new_location = self.model.random.choice(parcel_list)
                self.model.grid.move_agent(self, new_location)

        else:
            None

**Model class**

In [3]:
# Q: the whole happy sad blue red agent stuff, we can delete from here?

# define Model class
class Schelling(Model):
    """
    Model class for the Schelling segregation model.
    """

    def __init__(self, height=20, width=20, density=0.8, minority_pc=0.2):

        self.height = height
        self.width = width
        self.density = density
        self.minority_pc = minority_pc

        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, torus=False)

        self.parcel_values = {}

        ### helper functions for data collector
        def get_counts(self):
            values = dict()
            for cell_content, (x, y) in self.grid.coord_iter():
                values[(x,y)] = len(cell_content)
            return values

        def get_incomes(self):
            values = dict()
            for cell_content, (x, y) in self.grid.coord_iter():
                values[(x,y)] = [a.income for a in cell_content]
            return values
        
        ### data collector
        self.datacollector = DataCollector(
            model_reporters=
            {
                #"agent_incomes": count_agents(),
                "agent_counts":  lambda m: get_counts(m),
                "agent_incomes": lambda m: get_incomes(m),
            }, 
            agent_reporters=
                {
                    "x": lambda a: a.pos[0], 
                    "y": lambda a: a.pos[1]
                },
        )

        # Set up agents

        # The agent type is allocated using random instead of perfectly mixed
        agent_types_list = random.choices(
            population = [1,0], 
            weights = [1-minority_pc, minority_pc], 
            k = round(self.width * self.height * self.density))

        # Placing the agents on the grid (every cell has 10 agents at step 0)
        for _, (x,y) in self.grid.coord_iter():

            # randomly (with a probability of 0.8) a cell will be occupied
            # with exactly 10 agents
            if self.random.random() < self.density:
                for i in range(0, 10):
                    
                    agent_type = random.choice(agent_types_list) 
                    # Q: above we are drawing without replacement, this will lead to possibly different
                    # numbers of agents, do we want this, or do we rather 
                    # want to control exact minority:majority ratio?
                    
                    if agent_type == 1: # 1 is the majority
                        agent = SchellingAgent(
                            (x, y, i), # Q: 
                            (x, y), 
                            self, 
                            agent_type,
                            np.random.normal(loc=40, scale=5)
                            )  # Adding income distribution
                    else: # 0 is the minority
                        agent = SchellingAgent(
                            (x, y, i), 
                            (x, y), 
                            self, 
                            agent_type, 
                            np.random.normal(loc=100, scale=10)
                            )
                    self.grid.place_agent(agent=agent, pos=(x, y))
                    self.schedule.add(agent)

        self.running = True
        self.datacollector.collect(self)


    def step(self):
        """
        Run one step of the model. Update the average income in a parcel (NEW)
        """
        #Updating dictonary where (x,y) are the keys and the parcel's average income the value
        for list_agents, (x,y) in self.grid.coord_iter():
            
            income_list =[]

            if self.grid.is_cell_empty((x, y)):
                # or: if len(list_agents) == 0 ? 
                self.parcel_values[(x,y)] = 0

            else:
                for agent in list_agents:
                    income_list += [agent.income]

                parcel_value = statistics.mean(income_list)
                self.parcel_values[(x,y)] = parcel_value

        #print(self.parcel_values)

        self.schedule.step()
        # collect data
        self.datacollector.collect(self)

    def run_model(self, step_count = 50):
        for _ in range(step_count):
            self.step()
        print(f"Model run finished, number of steps: {step_count}")

**Running the model**

In [4]:
model = Schelling(
    height = 20, 
    width = 20, 
    density = 0.7, 
    minority_pc = 0.3)
total_steps = 50
model.run_model(total_steps)
# Q: we can ignore model.running because we have no stop condition built in?


Model run finished, number of steps: 50


In [5]:
df = model.datacollector.get_model_vars_dataframe()
df.head()

,agent_counts,agent_incomes
0,"{(0, 0): 10, (0, 1): 10, (0, 2): 0, (0, 3): 10...","{(0, 0): [43.93349877531341, 37.71560442268942..."
1,"{(0, 0): 8, (0, 1): 7, (0, 2): 10, (0, 3): 6, ...","{(0, 0): [43.93349877531341, 37.71560442268942..."
2,"{(0, 0): 6, (0, 1): 5, (0, 2): 3, (0, 3): 5, (...","{(0, 0): [37.71560442268942, 33.03191915180148..."
3,"{(0, 0): 11, (0, 1): 9, (0, 2): 2, (0, 3): 4, ...","{(0, 0): [37.71560442268942, 33.03191915180148..."
4,"{(0, 0): 7, (0, 1): 7, (0, 2): 2, (0, 3): 8, (...","{(0, 0): [37.71560442268942, 33.03191915180148..."


In [6]:
# find absolute max values for density and income

# max_income (if income stays the same across the entire model)
max_income = max([item for sublist in df.loc[0,"agent_incomes"].values() for item in sublist])
max_income = np.round(math.ceil(max_income/10))*10 # rounding up to nearest upper multiple of 10

# max agent density per cell
max_count = df.apply(lambda x: max(x.agent_counts.values()), axis = 1).max()
max_count = np.round(math.ceil(max_count/10))*10 # rounding up to nearest upper multiple of 10

In [7]:
os.makedirs("../results/model01/", exist_ok = True)

for mystep in range(total_steps):

    zfill_step = "{:03d}".format(mystep)
    print(zfill_step)

    fig, ax = plt.subplots(1,3, figsize = (30,10))

    vals_counts_incomes = {}

    # plot agent numbers
    i = 0
    vals_counts = np.zeros((model.width, model.height))
    for k, v in df.loc[mystep, "agent_counts"].items():
        vals_counts_incomes[k] = {}
        vals_counts_incomes[k]["count"] = v
        vals_counts[k]=v
    im = ax[i].imshow(
        vals_counts,
        vmin = 0,
        vmax = max_count,
        cmap = "Reds",
        )
    ax[i].set_title("Agent counts")
    ax[i].set_axis_off()
    plt.colorbar(im, shrink = 0.7)

    # plot agent mean incomes
    i = 1
    vals_incomes = np.zeros((model.width, model.height))
    for k, v in df.loc[mystep, "agent_incomes"].items():
        vals_counts_incomes[k]["income"] = v
        if v:
            vals_incomes[k]=statistics.median(v)
    ax[i].set_title("Income medians")
    im = ax[i].imshow(
        vals_incomes,
        vmin = 0, 
        vmax = max_income,
        cmap = "Greens"
        )
    ax[i].set_axis_off()
    plt.colorbar(im, shrink = 0.7)

    # plot agent counts vs. median incomes
    i = 2
    ax[i].scatter(
        x = [v["count"] for v in vals_counts_incomes.values()],
        y = [statistics.median(v["income"]) if v["income"] else 0 for v in vals_counts_incomes.values()],
        s = 5,
        color = "black"
    )
    ax[i].set_title("Agent count vs. median income")
    ax[i].set_xlim([0,max_count])
    ax[i].set_xlabel("Agent counts")
    ax[i].set_ylim([0,max_income])
    ax[i].set_ylabel("Agent incomes")
    plt.title(zfill_step)
    fig.savefig(f"../results/model01/{zfill_step}.png")

    plt.close()

000
001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049


In [8]:
fps = 1
img_folder_name = "../results/model01/"
images = sorted([img for img in os.listdir(img_folder_name) if img.endswith(".png")])
video_name = "test.mp4"
frame = cv2.imread(os.path.join(img_folder_name, images[0]))
height, width, layers = frame.shape
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, fps, (width,height))
for image in images:
    video.write(cv2.imread(os.path.join(img_folder_name, image)))
cv2.destroyAllWindows()
video.release()